In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split

from IPython.core.display import HTML # permet d'afficher du code html dans jupyter

Fonction pour tracer les courbes d'apprentissage

In [ ]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(est, X_train, y_train) :
    train_sizes, train_scores, test_scores = learning_curve(estimator=est, X=X_train, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
                                                        cv=5,
                                                        n_jobs=-1)
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    plt.figure(figsize=(8,10))
    plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='training accuracy')
    plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
    plt.plot(train_sizes, test_mean,color='green', linestyle='--',marker='s', markersize=5,label='validation accuracy')
    plt.fill_between(train_sizes,test_mean + test_std,test_mean - test_std,alpha=0.15, color='green')
    plt.grid(b='on')
    plt.xlabel('Number of training samples')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.ylim([0.6, 1.0])
    plt.show()

Fonction pour tracer la courbe ROC

In [ ]:
def plot_roc_curve(est,X_test,y_test) :
    probas = est.predict_proba(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.figure(figsize=(8,8))
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
    plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
    plt.xlim([-0.05,1.2])
    plt.ylim([-0.05,1.2])
    plt.ylabel('Taux de vrais positifs')
    plt.xlabel('Taux de faux positifs')
    plt.show

# Dataset de l'écriture Kannada MNIST

Tout d'abord, afin de réaliser ce notebook je me suis inspiré de celui sur les chiffres manuscrits réalisés en cours, de celui sur zalando ainsi que du notebook suivant: https://www.kaggle.com/marcovasquez/basic-ml-logistic-dt-pca-xgboost#7--Machine-Learning-Algorithm

L’écriture kannada, ou l’écriture kannara, est un alphasyllabaire de la famille de la brahmi principalement utilisé pour écrire le kannada, l’une des langues dravidiennes du sud de l’Inde parlée notamment dans l'état du Karnataka

In [ ]:
df = pd.read_csv("../input/Kannada-MNIST/Dig-MNIST.csv")

In [ ]:
df.shape

On a 785 colonnes :

une colonne 'label' identifiant la "lettre"
et 784 colonnes de pixels 

In [ ]:
df.head(10)

On crée la cible y (colonne 'label') :

In [ ]:
y = df['label']

et les caractéristiques X

In [ ]:
X = df.drop(['label'], axis=1)

On sépare les ensembles d'apprentissage et de test:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Nos ensembles sont prêts pour les différentes méthodes de machine learning, nous allons cependant visualiser les images dans un premier temps.

# Visualisation des images MNIST

Pour visualiser les images, on va convertir une ligne de 784 pixels en une matrice 28x28
Il faut en premier transformer le dataframe X en un tableau :

In [ ]:
X1 = np.array(X)

In [ ]:
print(X1[0])

On applique ensuite la méthode reshape pour convertir cette ligne en une matrice 28x28

In [ ]:
image = X1[0].reshape(28,28)
print(image)

On peut désormais afficher cette matrice sous forme d'image

In [ ]:
plt.imshow(image)

On peut l'afficher en niveaux de gris avec la colonne label comme titre

In [ ]:
plt.imshow(image, cmap="gray_r")
plt.axis('off')
plt.title(y[0])

In [ ]:
n_samples = len(df.index)
images = X1.reshape(n_samples,28,28)

On affiche les 50 premières lignes

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(images[i], cmap="gray_r")
    plt.title(y[i])

**Appliquons désormais les différentes méthodes de machine learning à la reconnaissance d'objets**

## Machine Learning : Random Forest

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
plot_learning_curve(rf, X, y)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

## Machine Learning : Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_lr = lr.predict(X_test)
cm = confusion_matrix(y_test, y_lr)
print(cm)
print(classification_report(y_test, y_lr))
score_lr = accuracy_score(y_lr, y_test)

In [ ]:
plot_learning_curve(lr, X, y)

## Machine Learning: XGBoost

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))
score_xgb = accuracy_score(y_xgb, y_test)

# Machine Learning: PCA svm

La méthode PCA consiste à transformer des variables liées entre elles (dites « corrélées » en statistique) en nouvelles variables décorrélées les unes des autres. Ces nouvelles variables sont nommées « composantes principales », ou axes principaux. Il s'agit d'une approche à la fois géométrique et statistique. Principalement utilisé comme outil de compresion linéaire, d'analyse de séries dynamiques d'images ou d'images multi spectrales.

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

https://medium.com/apprentice-journal/pca-application-in-machine-learning-4827c07a61db

https://en.wikipedia.org/wiki/Principal_component_analysis


Les machines à vecteurs de support ou séparateurs à vaste marge sont un ensemble de techniques d'apprentissage supervisé destinées à résoudre des problèmes de discrimination et de régression.

In [ ]:
from sklearn import svm
from sklearn.decomposition import PCA
pca = PCA(n_components=0.7,whiten=True)
X_train_PCA = pca.fit_transform(X_train)
X_test_PCA = pca.transform(X_test)
sv = svm.SVC(kernel='rbf',C=9)
sv.fit(X_train_PCA , y_train)
y_pca = sv.predict(X_test_PCA)
cm = confusion_matrix(y_test, y_pca)
print(cm)
print(classification_report(y_test, y_pca))
score_pca = accuracy_score(y_pca, y_test)

## Machine Learning: Decision Tree

C'est une technique d'apprentissage supervisé : on utilise un ensemble de données pour lesquelles on connaît la valeur de la variable-cible afin de construire l'arbre (données dites étiquetées), puis on extrapole les résultats à l'ensemble des données de test. 

In [ ]:
from sklearn import tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
y_dt = dt.predict(X_test)

cm = confusion_matrix(y_test, y_dt)
print(cm)
print(classification_report(y_test, y_dt))
score_dt = accuracy_score(y_dt, y_test)

# Visualisation des scores des différents algorithmes

In [ ]:
models = pd.DataFrame({
    'Model': ['LogisticRegression','XGBOOST', 'PCA', 'Decision Tree',
              ],
    'Score': [score_lr,score_xgb,score_pca,score_dt]})
models.sort_values(by='Score', ascending=False)

In [ ]:
plt.subplots(figsize =(10, 5))

sns.barplot(x='Score', y = 'Model', data = models, palette="Set3")

plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)')
plt.ylabel('Algorithm')

On remarque que la méthode PCA est la plus précise avec 0.93 de précision. 